In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
#OS
import os

#Data handling
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


In [ ]:
trainPath = 